In [1]:
import numpy as np
from hmmlearn import hmm
import csv


In [2]:
hmm_states = 2

model = hmm.CategoricalHMM(n_components=hmm_states)
model.emissionprob_ = np.array([[0.1,0.9],[0.9,0.1]])
model.transmat_ = np.array([[0.9,0.1],[0.1,0.9]])
model.startprob_ = np.array([0.8,0.2])
data,states = model.sample(n_samples = 1000,random_state=28)
data = np.squeeze(data)

In [29]:
print(states)

[0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 

In [4]:

# constant n_obs
# n_obs = 1000
# # Set p(H) apriori for simulation as a biased coin.
# p_h = 0.8

# # Model experiment as a single biased coin flipped 1000 times.
# data = np.random.binomial(1, p_h, n_obs=n_obs)

In [136]:
n_states = 2
n_features = 2
n_obs = data.shape[0]
# emission = np.random.rand(n_obs,n_states)
# emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))

emission = np.random.rand(n_states,n_features)
emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))
# emission = np.array([[0.5,0.5],[0.5,0.5]])

transition =  np.random.rand(n_states,n_states)
transition = transition/np.tile(np.expand_dims(np.sum(transition,axis=1),axis=1),(1,2))
transition = np.array([[0.92,0.08],[0.1,0.9]])


forward = np.random.rand(n_obs,n_states)
scale_factors = np.zeros((n_obs))
forward_hat = np.zeros((n_obs,n_states))

backward = np.random.rand(n_obs,n_states)
backward_hat = np.zeros((n_obs,n_states))

init_prob = np.array([0.5,0.5])


p_old = -10000
tol = 0.0001
max_iter = 10

mu = np.random.rand(n_states)

In [137]:
print(f'emission is {emission} and transition is {transition}')

emission is [[0.52895737 0.47104263]
 [0.30210931 0.69789069]] and transition is [[0.92 0.08]
 [0.1  0.9 ]]


In [138]:
for ite in range(max_iter):
    forward[0,:] = init_prob * emission[:,data[0]]
    scale_factors[0] = np.sum(forward[0,:])
    forward_hat[0,:] = forward[0,:]/scale_factors[0]
    
    for t in range(n_obs-1):
        temp = np.matmul(forward_hat[t,:] ,np.transpose(transition)) * emission[:,data[t+1]]
        scale_factors[t+1] = np.sum(temp)
        forward_hat[t+1,:] = temp/scale_factors[t+1]
        # print(f'temp is {temp} and the scale factor is {scale_factors[t+1]} and the forward_hat is {forward_hat[t+1]}')

        forward[t+1,:] = forward_hat[t+1,:]*np.prod(scale_factors[0:t+1])

    backward[-1,:] = 1
    backward_hat[-1,:] = backward[-1,:]
    for t in reversed(range(n_obs-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[:,data[t+1]],transition)
        backward_hat[t,:] = temp/scale_factors[t+1]
        backward[t,:] = backward_hat[t,:]*np.prod(scale_factors[t+1:-1])


    a = np.zeros((n_obs,n_states))
    b = np.zeros((n_obs,n_states,n_states))
    for i in range(n_obs):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
    for t in range(n_obs-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])
            # print(np.sum(b[0:-1,i,j]),np.sum(b[0:-1,i,:]))

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])

    for i in range(n_states):
        for j in range(n_states):
            emission[j,i] = np.sum(a[np.argwhere(data==i),j]) / np.sum(a[:,j])
            
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p},transition is {transition}')
    # print(f'transition prob is: {transition}')
    # print(f'emission prob is:{emission}')
    if p>p_old and p - p_old < tol:
        break
    p_old = p

p is:-660.4496346855619,transition is [[0.9390847  0.0609153 ]
 [0.09555603 0.90444397]]
p is:-588.2384251946714,transition is [[0.96653978 0.03346022]
 [0.07362996 0.92637004]]
p is:-564.173172733355,transition is [[0.98683269 0.01316731]
 [0.05871576 0.94128424]]
p is:-579.1065309313738,transition is [[0.99565698 0.00434302]
 [0.0826504  0.9173496 ]]
p is:-604.6590225874478,transition is [[0.99896394 0.00103606]
 [0.26350603 0.73649397]]
p is:-689.1666118061696,transition is [[9.99888786e-01 1.11214197e-04]
 [8.55705920e-01 1.44294080e-01]]
p is:-831.4632957829967,transition is [[9.99999937e-01 6.33693493e-08]
 [9.99708342e-01 2.91657760e-04]]
p is:-698.6505182034773,transition is [[1.00000000e+00 2.27361753e-11]
 [9.99999510e-01 4.89794627e-07]]
p is:-691.9310064762531,transition is [[1.00000000e+00 8.09011703e-15]
 [9.99999999e-01 8.28998526e-10]]
p is:-691.9309038649061,transition is [[1.00000000e+00 2.87866451e-18]
 [1.00000000e+00 1.40311846e-12]]


In [127]:
print(b[0:-1,1,1])

[2.27307924 0.25773233 0.25773233 0.25773233 0.25773233 0.02748279
 2.27307924 0.02748279 0.24238543 0.24238543 0.24238543 2.27307924
 0.02748279 0.24238543 2.27307924 0.25773233 0.25773233 0.25773233
 0.25773233 0.25773233 0.25773233 0.02748279 2.27307924 0.25773233
 0.25773233 0.25773233 0.02748279 0.24238543 0.24238543 0.24238543
 0.24238543 0.24238543 0.24238543 0.24238543 0.24238543 0.24238543
 2.27307924 0.25773233 0.25773233 0.25773233 0.25773233 0.25773233
 0.25773233 0.25773233 0.25773233 0.25773233 0.25773233 0.25773233
 0.25773233 0.25773233 0.02748279 2.27307924 0.25773233 0.25773233
 0.25773233 0.25773233 0.25773233 0.25773233 0.25773233 0.25773233
 0.25773233 0.25773233 0.25773233 0.25773233 0.25773233 0.25773233
 0.25773233 0.25773233 0.25773233 0.25773233 0.25773233 0.25773233
 0.25773233 0.02748279 2.27307924 0.25773233 0.25773233 0.25773233
 0.25773233 0.25773233 0.02748279 0.24238543 0.24238543 0.24238543
 2.27307924 0.25773233 0.25773233 0.25773233 0.02748279 0.2423

In [128]:
transition

array([[9.69894213e-14, 1.00000000e+00],
       [9.76588765e-20, 1.00000000e+00]])

In [123]:
emission

array([[0.25860591, 0.74139409],
       [0.94555583, 0.05444417]])

In [30]:
# model = hmm.CategoricalHMM(n_components=n_states,emissionprob_prior=emission,transmat_prior=transition,startprob_prior=init_prob)
model2 = hmm.CategoricalHMM(n_components=n_states,n_iter=1000)


In [31]:
model2.fit(data.reshape(-1,1))
hidden_states = model2.predict(data.reshape(-1,1))
print("Most likely hidden states:", hidden_states)


Most likely hidden states: [0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 

In [40]:
model2.monitor_.converged


True

In [33]:
model2.emissionprob_

array([[0.08646279, 0.91353721],
       [0.87767051, 0.12232949]])

In [34]:
model2.transmat_

array([[0.88572501, 0.11427499],
       [0.10842401, 0.89157599]])